In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import quote


In [20]:
# Input Search Query
SEARCH_QUERY = 'tv'

In [21]:
PROXY_CRAWL_TOKEN = '8wJqpL2FErGiPGX7QWP6Aw'

In [22]:
def get_proxy(url):
    quoted_url = quote(url)
    res = requests.get(f'https://api.proxycrawl.com/?token={PROXY_CRAWL_TOKEN}&url={quoted_url}')
    res.raise_for_status() # Raise error if it fails
    return res.content

In [23]:
#### A function to get the content of the page of required query
def search_in_amazon(search_query):
    url = f"https://www.amazon.com/s?k={search_query}"
    return get_proxy(url)

#### A function to get the contents of individual product pages using 'data-asin' number (unique identification number)
def search_asin(asin):
    url = f"https://www.amazon.com/dp/{asin}"
    return get_proxy(url)

#### A function to pass on the link of 'see all reviews' and extract the content
def search_reviews(review_link):
    url = "https://www.amazon.com{review_link}"
    return get_proxy(url)


### Product Name extraction

In [14]:
print("Start to extract product names")

product_names = []
data_asin = []
LAST_PAGE = 21
for i in range(1,LAST_PAGE):
    print(f"Iteration {i}/{LAST_PAGE}")
    html = search_in_amazon(SEARCH_QUERY+'&page='+str(i))
    soup = BeautifulSoup(html)
    for i in soup.findAll("span",{'class':'a-size-medium a-color-base a-text-normal'}):
        product_names.append(i.text) # adding the product names to the list

    for i in soup.findAll("div", {"class":"s-result-item"}):
        if i['data-asin']:
            data_asin.append(i['data-asin'])

print("Finished extract product names")
print(product_names)

Start to extract product names
Iteration 1/21
Iteration 2/21
Iteration 3/21
Iteration 4/21
Iteration 5/21
Iteration 6/21
Iteration 7/21
Iteration 8/21
Iteration 9/21
Iteration 10/21
Iteration 11/21
Iteration 12/21
Iteration 13/21
Iteration 14/21
Iteration 15/21
Iteration 16/21
Iteration 17/21
Iteration 18/21
Iteration 19/21
Iteration 20/21
Finished extract product names
['TCL 32S325 32 Inch 720p Roku Smart LED TV (2019)', 'Samsung UN65RU7100FXZA Flat 65-Inch 4K UHD 7 Series Ultra HD Smart TV with HDR and Alexa Compatibility (2019 Model)', 'TCL 43S425  43 Inch 4K Ultra HD Smart Roku LED TV (2018)', 'LG 65UM7300PUA Alexa Built-in 65" 4K Ultra HD Smart LED TV (2019)', 'Samsung Electronics 4K Smart LED TV (2018), 55" (UN55NU6900FXZA)', 'TOSHIBA 50LF711U20 50-inch 4K Ultra HD Smart LED TV HDR - Fire TV Edition', 'Samsung UN55RU8000FXZA Flat 55-Inch 4K 8 Series Ultra HD Smart TV with HDR and Alexa Compatibility (2019 Model)', 'Samsung QN43Q60RAFXZA Flat 43-Inch QLED 4K Q60 Series Ultra HD Sm

In [18]:
len(product_names)

112

When scrawling the all pages of product list in specific search query, I could discover that there are same products in the list. <br>
Therefore, I needed to remove the same product in the list of ASIN.

In [7]:

asin_list = list(dict.fromkeys(data_asin) )
data_asin = asin_list

link=[]
for i in range(len(data_asin)):
    print(i)
    html = search_asin(data_asin[i])
    soup = BeautifulSoup(html)
    for i in soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"}):
        print(i['href'])
        link.append(i['href'])

NameError: name 'data_asin' is not defined

In [8]:
search_query_list = []
reviews=[]


In [ ]:


for j in range(len(link)):
    print(j, 'th started')
    for k in range(1, 2):
        html = search_reviews(link[j]+'&pageNumber='+str(k))
        soup = BeautifulSoup(html)
        if soup.find('div',
                     {"class" : "a-section a-spacing-top-large a-text-center no-reviews-section"}):
            print('No review, Pass')
            break
        else:
            print('There is review')
#           for i in soup.findAll("span",{'data-hook':"review-body"}):
            for i in soup.findAll("span",{'data-hook':"a-size-base review-text review-text-content"}):
                reviews.append(i.text)
                search_query_list.append(search_query)

In [ ]:
rev={'search_query':search_query_list, 'reviews' :reviews} #converting the reviews list into a dictionary
review_data=pd.DataFrame.from_dict(rev) #converting this dictionary into a dataframe

df = review_data.replace('\n','', regex=True)

writer= pd.ExcelWriter(SEARCH_QUERY+'_review.xlsx')
df.to_excel(writer, 'Sheet1', index=False)
writer.save()